In [3]:
import os
import geopandas as gpd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import elevation
import pandas as pd
import folium as g

# 전처리

##### * 사고포인트 지도와 결합
##### * 서울시 dem으로 경사도 래스터 파일로 변환

# 분석 순서

#### 1.사고포인트 folium 사용하여 서울지도에 마크
#### 2.서울시 dem -> 경사 래스터로 변환
###### -> richdem 사용하여 경사도로 표현되는 래스터 파일 생성
#### 3.사고포인트와 경사 래스터 결합
##### -> rasterio 이용하여 결합
#### 4.사고포인트의 경사도 분석
###### -> zonal statics 사용
#### 5.사고포인트의 경사도에 따른 가중치 할당
###### -> 경사도 0 ~ 1도, 1~ 2도, 2 ~ 4도, 4 ~ 6도 총 4단계 구간으로 나눔. 0 ~ 1도 구간에 사고 데이터 50% 이상 존재. 이를 이용하여 각각의 데이터프레임을 생성하였다. 이후 weighted_value에 각각 가중치 1, 1.25, 1.5, 2 할당함. 가중치가 다른 4개의 데이터프레임을 ave_seoulslope_accident_result의 데이터로 합침
#### 6.사고포인트의 경사도 값 그래프화

# 데이터명
##### '사고포인트 좌표csv'
   ###### -> accidentpoint11.csv
##### '사고포인트 포함 서울지도(html)'
   ###### -> seoul
##### '서울시dem'
   ###### -> dem
##### '서울시 경사도 래스터'
   ###### -> seoulslope
##### '경사도 기준 가중치 dataframe'
   ###### -> ave_seoulslope_accident1, ave_seoulslope_accident2, ave_seoulslope_accident3, ave_seoulslope_accident4
##### '경사도 기준 가중치 포함 총 dataframe'
   ###### -> ave_seoulslope_accident_result
##### '경사도 그래프'
   ###### -> ave_seoulslope_accident_bar

# 사고포인트 DataFrame 생성

In [ ]:
#accidentpoint11.csv(사고포인트좌표) 이용하여
#사고포인트dataframe(df) 생성
df = pd.read_csv("accidentpoint11.csv",\
                 encoding="euc-kr")
df = gpd.GeoDataFrame(
    df[['위도', '경도']],
    geometry=gpd.points_from_xy(df.위도, df.경도))
df.head()

# Folium 기반 지도 위 사고포인트 시각화

In [ ]:
#서울 지도생성(seoul=>서울 지도)
seoul = g.Map(location=[37.55,126.98],tiles='Stamen Terrain', zoom_start=12)

# 서울 지역구 생성
import requests
import json

## 서울 행정구역 json raw파일(githubcontent)불러오기
r = requests.get('https://is.gd/cKFymA')
c = r.content
seoul_geo = json.loads(c)
g.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(seoul)

# df 기반 사고포인트 mark
df.reset_index(inplace=True)
for i in range(len(df)):
    point=g.Marker([df.loc[i]['위도'],df.loc[i]['경도']],\
                   popup=df.loc[i]['geometry'],\
                   icon=g.Icon(color='darkgreen'))
    point.add_to(seoul)
    
seoul

In [ ]:
#서울 지도에 사고 포인트 올리기_지도생성(seoul=서울부분지도)
seoul = g.Map(location=[37.55,126.98],tiles='Stamen Terrain', zoom_start=12)
seoul

In [ ]:
#서울 지도생성(seoul=>서울 지도)
seoul = g.Map(location=[37.55,126.98],tiles='Stamen Terrain', zoom_start=12)

# 서울 지역구 생성
import requests
import json

## 서울 행정구역 json raw파일(githubcontent)불러오기
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)
g.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(seoul)

# df 기반 사고포인트 mark
df.reset_index(inplace=True)
for i in range(len(df)):
    point=g.Marker([df.loc[i]['위도'],df.loc[i]['경도']],\
                   popup=df.loc[i]['geometry'],\
                   icon=g.Icon(color='darkgreen'))
    point.add_to(seoul)
    
seoul

In [ ]:
#seoul로 지도 저장
seoul.save('pointonmap.html')

# 2. 서울시 DEM 기반 사고포인트 경사도 분석

In [ ]:
# dem1(서울dem)을 dem으로 생성
dem = rio.open('dem1.tif') 
dem_array = dem.read(1).astype('float64')

fig, ax = plt.subplots(1, figsize=(18, 9))
show(dem_array, cmap='PuBuGn_r', ax=ax)
plt.axis("off")
plt.show()

## 2-1. 경사도 래스터 생성

In [ ]:
# richdem 모듈 사용하여 경사 분석
dem_richdem = rd.rdarray(dem_array, no_data=-9999)

dem_slope = rd.TerrainAttribute(dem_richdem, attrib='slope_degrees')
rd.rdShow(dem_slope, axes=False, cmap='PuBuGn', figsize=(18, 8));

## 2-2. 경사래스터와 사고포인트 결합

In [ ]:
# 사고포인트가 위경도 좌표인 4326이므로 래스터와 좌표 맞추기(5181로!)
from pyproj import Transformer, transform

# 좌표 변환 (4326 -> 5181)

transformer_4326_5186 = Transformer.from_proj(4326,5181)
transformed_x, transformed_y = [], []

for coords in df['geometry']:
    transformed_coords = transformer_4326_5186.transform(coords.x, coords.y)
    # 각 geometry 데이터에서 .x, .y 메서드를 사용하면 x, y 좌표를 얻을 수 있음
    
    transformed_x.append(transformed_coords[1])
    transformed_y.append(transformed_coords[0])
    
df.drop(['geometry'], axis=1, inplace=True)    # 기존 geometry 컬럼 삭제
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(transformed_x, transformed_y))

In [ ]:
# 서울의 경사도 래스터 파일 = seoulslope
seoulslope = rio.open('seoulslope.tif')
seoulslope

In [ ]:
# seoulslope위에 사고포인트(df) 결합
fig, ax = plt.subplots(1,1,figsize=(10,10))

# transform rasterio plot to real world coords
extent=[seoulslope.bounds[0], seoulslope.bounds[2], seoulslope.bounds[1], seoulslope.bounds[3]]
ax = rio.plot.show(seoulslope, extent=extent, ax=ax, title='point slope')

df.plot(ax=ax, facecolor='red')

## 2-3. 사고포인트의 경사도 분석 및 가중치 할당

#### 가중치는 경사도 0.636598, 1.600029, 5.215480 구간으로 나누어 각각 1.25, 1.5, 2로 할당(각 구간마다 사고 20개씩 존재)

In [ ]:
# 결합된 포인트의 경사도 DataFrame 생성('mean') 
ave_seoulslope=zonal_stats(df, slope_array, affine=affine, stats = ['mean'],geojson_out=True)

#extracting the average slope data from the list
avg_slope = []
i =0

while i < len(ave_seoulslope):
    avg_slope.append(ave_seoulslope[i]['properties'])
    i=i+1

# ave_seoulslope_accident 
ave_seoulslope_accident=pd.DataFrame(avg_slope)
ave_seoulslope_accident

ave_seoulslope_accident.sort_values('mean',ascending=False)

In [ ]:
ave_seoulslope_accident

In [ ]:
pm_slope = pd.read_csv('ave_seoulslope_accident.csv')
# 경사도 히스토그램
plt.title('slope')
plt.hist(pm_slope['mean'], color = 'g', alpha = 0.5)

In [ ]:
# 경사도 <=1의 데이터프레임
ave_seoulslope_accident1=ave_seoulslope_accident\
[ (ave_seoulslope_accident['mean']<=1)]
ave_seoulslope_accident1

In [ ]:
# 경사도 <=1의 데이터프레임의 가중치 할당
ave_seoulslope_accident1['weighted_value']=2
ave_seoulslope_accident1

In [ ]:
# 1< 경사도 <=2의 데이터프레임
ave_seoulslope_accident2=ave_seoulslope_accident[ (ave_seoulslope_accident['mean']>1) & (ave_seoulslope_accident['mean'] <= 2)]
ave_seoulslope_accident2

In [ ]:
# 1< 경사도 <=2의 데이터프레임의 가중치 할당
ave_seoulslope_accident2['weighted_value']=1.5
ave_seoulslope_accident2

In [ ]:
# 2< 경사도 <=4의 데이터프레임
ave_seoulslope_accident3=ave_seoulslope_accident[ (ave_seoulslope_accident['mean']>2) & (ave_seoulslope_accident['mean'] <= 4)]
ave_seoulslope_accident3

In [ ]:
# 2< 경사도 <=4의 데이터프레임의 가중치 할당
ave_seoulslope_accident3['weighted_value']=1.25
ave_seoulslope_accident3

In [ ]:
# 4< 경사도 <=6의 데이터프레임
ave_seoulslope_accident4=ave_seoulslope_accident[ (ave_seoulslope_accident['mean']>4) & (ave_seoulslope_accident['mean'] <=6 )]
ave_seoulslope_accident4

In [ ]:
# 4< 경사도 <=6의 데이터프레임의 가중치 할당
ave_seoulslope_accident4['weighted_value']=1
ave_seoulslope_accident4

In [ ]:
#경사도 가중치 데이터프레임 결합
ave_seoulslope_accident_result = pd.concat\
([ave_seoulslope_accident1,ave_seoulslope_accident2,\
  ave_seoulslope_accident3,ave_seoulslope_accident4],\
 ignore_index=True)

ave_seoulslope_accident_result=ave_seoulslope_accident_result.\
rename(columns={'index':'사고위치순서'})
ave_seoulslope_accident_result.head()

## 2-4. 사고포인트의 평균 경사도

In [ ]:
#내림차순
ave_seoulslope_accident_bar = ave_seoulslope_accident.groupby(["index"])["mean"].\
mean().sort_values(ascending = False)
ave_seoulslope_accident_bar.plot.bar( figsize = (23,7), xlabel='location(index)'
                                     ,ylabel='mean of slope',y='mean' ,width=0.8,color='darkgreen')

seoulyongdo = '서울용도지역.shp'
seoul_yongdo = gpd.read_file(seoulyongdo,encoding='cp949')
seoul_yongdo=seoul_yongdo.to_crs(4326)
seoul_yongdo

In [ ]:
ax=seoul_yongdo.plot(figsize=(10,10))

In [ ]:
ax = seoul_yongdo.plot(column="DGM_NM", figsize=(10,10), alpha=0.8)
df.plot(ax=ax, marker='v', color='red', label='accidentpoint')
ax.set_title("Accident points on District of Seoul", fontsize=20)
ax.set_axis_off()
plt.legend()
plt.show()